In [1]:
import requests
import json
import re
province_list = [
    '北京市', '天津市', '河北省', '山西省', '内蒙古自治区', '辽宁省', '吉林省', '黑龙江省',
    '上海市', '江苏省', '浙江省', '安徽省', '福建省', '江西省', '山东省', '河南省',
    '湖北省', '湖南省', '广东省', '广西壮族自治区', '海南省', '重庆市', '四川省',
    '贵州省', '云南省', '西藏自治区', '陕西省', '甘肃省', '青海省', '宁夏回族自治区',
    '新疆维吾尔自治区', '台湾省', '香港特别行政区', '澳门特别行政区'
]

def extract_chinese_name(tags):
    # 提取中文名称
    if 'name:zh' in tags:
        return tags['name:zh']
    if 'name' in tags:
        # 提取连续的中文字符作为名称
        match = re.findall(r'[\u4e00-\u9fff]+', tags['name'])
        return ''.join(match) if match else tags['name']
    return "未知"

In [2]:
# 首先查询中国下面所有 行政区 并把香港 澳门 台湾 补入
country = "中国"
url = "https://overpass-api.de/api/interpreter"

# 查询中国的省级行政区（admin_level=4）
query = f"""
[out:json][timeout:25];
area["name"="{country}"]["admin_level"="2"]->.searchArea;
(
  relation(area.searchArea)["boundary"="administrative"]["admin_level"="4"];
);
out tags;
"""

response = requests.get(url, params={'data': query})
data = response.json()

provinces = set()
for element in data['elements']:
    name = extract_chinese_name(element['tags'])
    provinces.add((name, element['id']))

# 补充 澳门、
provinces.add(('澳門', 1867188))
provinces.add(('香港', 913110))
provinces.add(('台湾', 449220))
print(f"{country}下一级行政区有：")
for province in sorted(provinces):
    print(province)


中国下一级行政区有：
('上海市', 913067)
('云南省', 913094)
('内蒙古自治区', 161349)
('北京市', 912940)
('台湾', 449220)
('吉林省', 198590)
('四川省', 913068)
('天津市', 912999)
('宁夏回族自治区', 913101)
('安徽省', 913011)
('山东省', 913006)
('山西省', 913105)
('广东省', 911844)
('广西壮族自治区', 286342)
('新疆维吾尔自治区', 153310)
('江苏省', 913012)
('江西省', 913109)
('河北省', 912998)
('河南省', 407492)
('浙江省', 553302)
('海南省', 2128285)
('湖北省', 913106)
('湖南省', 913073)
('澳門', 1867188)
('甘肃省', 153314)
('福建省', 553303)
('西藏自治区', 153292)
('贵州省', 286937)
('辽宁省', 912942)
('重庆市', 913069)
('陕西省', 913100)
('青海省', 153269)
('香港', 913110)
('黑龙江省', 199073)


In [3]:
# 为每个省级迭代查询其下一级行政区 并且保存id
base_id = 3600000000
def get_admin6_list(id):
    id = base_id + id
    query = f"""
      [out:json][timeout:25];
    area({id})->.searchArea;
    (
      relation(area.searchArea)["boundary"="administrative"]["admin_level"="6"];
    );
    out tags;
    """
    response = requests.get(url, params={'data': query})
    data = response.json()

    admin6 = set()
    for element in data['elements']:
        name = extract_chinese_name(element['tags'])
        admin6.add((name, element['id']))
        
    return admin6

table_data = {}
id_data = {}
for province in province_list:
    table_data[province] = []
    id_data[province] = {}

for province, p_id in provinces:
    if province in ['台湾省', '香港特别行政区', '澳门特别行政区']:
        continue
    if "市" in province:
        # 地级市
        # 不查询
        table_data[province].append(province)
        continue
    p_id += base_id
    print(f"正在查询 {province} 的地级市...")
    query = f"""
    [out:json][timeout:25];
    area({p_id})["admin_level"="4"]->.searchArea;
    (
      relation(area.searchArea)["boundary"="administrative"]["admin_level"="5"];
    );
    out tags;
    """
    
    response = requests.get(url, params={'data': query})
    data = response.json()
    
    cities = set()
    for element in data['elements']:
        name = extract_chinese_name(element['tags'])
        cities.add((name, element['id']))
    
    # 名字短的先放，长的后放
    for city in sorted(cities, key=lambda x: (len(x[0]), x[0])):
        # print(city)
        admin6 = get_admin6_list(city[1])
        if admin6:
            # 有下级行政区
            table_data[province].append(city[0])
            id_data[province][city[0]] = city[1]

# 补充一下 直辖市 4个
id_data["北京市"] = {"北京市": 912940}
id_data["天津市"] = {"天津市": 912999}
id_data["上海市"] = {"上海市": 913067}
id_data["重庆市"] = {"重庆市": 931069}


正在查询 安徽省 的地级市...
正在查询 澳門 的地级市...
正在查询 宁夏回族自治区 的地级市...
正在查询 浙江省 的地级市...
正在查询 广东省 的地级市...
正在查询 香港 的地级市...
正在查询 河南省 的地级市...
正在查询 内蒙古自治区 的地级市...
正在查询 贵州省 的地级市...
正在查询 河北省 的地级市...
正在查询 黑龙江省 的地级市...
正在查询 四川省 的地级市...
正在查询 甘肃省 的地级市...
正在查询 云南省 的地级市...
正在查询 广西壮族自治区 的地级市...
正在查询 江西省 的地级市...
正在查询 台湾 的地级市...
正在查询 江苏省 的地级市...
正在查询 湖南省 的地级市...
正在查询 辽宁省 的地级市...
正在查询 山西省 的地级市...
正在查询 陕西省 的地级市...
正在查询 湖北省 的地级市...
正在查询 海南省 的地级市...
正在查询 山东省 的地级市...
正在查询 青海省 的地级市...
正在查询 福建省 的地级市...
正在查询 新疆维吾尔自治区 的地级市...
正在查询 西藏自治区 的地级市...
正在查询 吉林省 的地级市...


In [7]:
# 补充一下 香港 澳门 台湾
xianggang_id = 913110 + base_id
aomen_id = 1867188 + base_id
taiwan_id = 449220 + base_id

# 查询中国的省级行政区（admin_level=4）
query = f"""
[out:json][timeout:25];
area({taiwan_id})["admin_level"="2"]->.searchArea;
(
  relation(area.searchArea)["boundary"="administrative"]["admin_level"="4"];
);
out tags;
"""

response = requests.get(url, params={'data': query})
data = response.json()

provinces = set()
for element in data['elements']:
    name = extract_chinese_name(element['tags'])
    provinces.add((name, element['id']))

# 补充 台湾
for province in provinces:
    table_data["台湾省"].append(province[0])
    id_data["台湾省"][province[0]] = province[1]

id_data["香港特别行政区"] = {
    "九龍": 10268797,
    "新界": 10268964,
    "香港島": 10264792
}
id_data["澳门特别行政区"] = {
    '氹仔': 5758868,
    '路環': 5758869
}

table_data["香港特别行政区"] = list(id_data["香港特别行政区"].keys())
table_data["澳门特别行政区"] = list(id_data["澳门特别行政区"].keys())

In [8]:
print("查询完成！以下是各省的地级市列表：")
for province, cities in table_data.items():
    print(f"{province}：{', '.join(cities)}")

查询完成！以下是各省的地级市列表：
北京市：北京市
天津市：天津市
河北省：保定市, 唐山市, 廊坊市, 承德市, 沧州市, 衡水市, 邢台市, 邯郸市, 张家口市, 石家庄市, 秦皇岛市, 雄安新区
山西省：临汾市, 吕梁市, 大同市, 太原市, 忻州市, 晋中市, 晋城市, 朔州市, 运城市, 长治市, 阳泉市
内蒙古自治区：乌海市, 兴安盟, 包头市, 赤峰市, 通辽市, 阿拉善盟, 乌兰察布市, 呼伦贝尔市, 呼和浩特市, 巴彦淖尔市, 鄂尔多斯市, 锡林郭勒盟
辽宁省：丹东市, 大连市, 抚顺市, 朝阳市, 本溪市, 沈阳市, 盘锦市, 营口市, 辽阳市, 铁岭市, 锦州市, 阜新市, 鞍山市, 葫芦岛市
吉林省：吉林市, 四平市, 松原市, 白城市, 白山市, 辽源市, 通化市, 长春市, 延边朝鲜族自治州
黑龙江省：伊春市, 大庆市, 绥化市, 鸡西市, 鹤岗市, 黑河市, 七台河市, 佳木斯市, 双鸭山市, 哈尔滨市, 牡丹江市, 齐齐哈尔市, 大兴安岭地区
上海市：上海市
江苏省：南京市, 南通市, 宿迁市, 常州市, 徐州市, 扬州市, 无锡市, 泰州市, 淮安市, 盐城市, 苏州市, 镇江市, 连云港市
浙江省：丽水市, 台州市, 嘉兴市, 宁波市, 杭州市, 温州市, 湖州市, 绍兴市, 衢州市, 金华市, 舟山市（舟山群岛新区）
安徽省：亳州市, 六安市, 合肥市, 安庆市, 宣城市, 宿州市, 池州市, 淮北市, 淮南市, 滁州市, 芜湖市, 蚌埠市, 铜陵市, 阜阳市, 黄山市, 马鞍山市
福建省：三明市, 南平市, 厦门市, 宁德市, 泉州市, 漳州市, 福州市, 莆田市, 龙岩市
江西省：上饶市, 九江市, 南昌市, 吉安市, 宜春市, 抚州市, 新余市, 萍乡市, 赣州市, 鹰潭市, 景德镇市
山东省：东营市, 临沂市, 威海市, 德州市, 日照市, 枣庄市, 泰安市, 济南市, 济宁市, 淄博市, 滨州市, 潍坊市, 烟台市, 聊城市, 菏泽市, 青岛市
河南省：信阳市, 南阳市, 周口市, 商丘市, 安阳市, 开封市, 新乡市, 洛阳市, 漯河市, 濮阳市, 焦作市, 许昌市, 郑州市, 鹤壁市, 三门峡市, 平顶山市, 驻马店市
湖北省：十堰市, 咸宁市, 孝感市, 宜昌市, 武汉市, 荆州市, 荆门市, 襄阳市, 鄂州市,

In [10]:
# 保存
data = {
    "name": table_data,
    "id": id_data
}
with open('china_admin_data_1.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)